In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os
import shutil
import logging
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import _load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline


In [18]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = _load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


Number of conformers generated: 4
Energies: ['-23.275472740000001', '-23.273513550000001', '-23.273084699999998', '-23.27281279']


In [19]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
import os
os.getcwd()

'/home/ergot/linux_qm/notebooks/.orca_tmp/8adf06e6-c213-4011-a9c1-a98e31834d10'

In [21]:
logging.getLogger().setLevel(logging.DEBUG)

In [22]:
from linux_qm.qm.orca.orca import OrcaDriver
orca = OrcaDriver()

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'HF def2-SVP'
orca.single_point(conf)

[24-11-2023 01:23:30] [DEBUG] ORCA INPUT:
!HF def2-SVP   
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71485383          0.42879861         -0.15401719
O            1.61470498         -0.39693278         -0.44574932
C            0.56782258         -0.27290508          0.47463299
C           -0.66124326         -1.11989942          0.13118524
N           -1.51499641          0.04690380          0.36976532
C           -2.58130515          0.29499435         -0.56441429
C           -0.36189400          0.95006260          0.33945427
H            2.43980715          1.48947558         -0.18100099
H            3.12947682          0.19813400          0.83513400
H            3.46307851          0.22800321         -0.91803329
H            0.93020169         -0.41943851          1.50068704
H           -0.88479495         -1.96628603          0.78047677
H           -0.62918791         -1.44665669         -0.91402471
H           -3.06204822          1.23783150   

In [23]:
orca.geometry_optimization(conf)

[24-11-2023 01:23:33] [INFO] Geometry optimization
[24-11-2023 01:23:33] [INFO] Method: HF def2-SVP
[24-11-2023 01:23:33] [DEBUG] ORCA INPUT:
!HF def2-SVP OPT  
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71485383          0.42879861         -0.15401719
O            1.61470498         -0.39693278         -0.44574932
C            0.56782258         -0.27290508          0.47463299
C           -0.66124326         -1.11989942          0.13118524
N           -1.51499641          0.04690380          0.36976532
C           -2.58130515          0.29499435         -0.56441429
C           -0.36189400          0.95006260          0.33945427
H            2.43980715          1.48947558         -0.18100099
H            3.12947682          0.19813400          0.83513400
H            3.46307851          0.22800321         -0.91803329
H            0.93020169         -0.41943851          1.50068704
H           -0.88479495         -1.96628603          0.78047677
H      

In [24]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'THF'
orca.options['n_jobs'] = 16

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 01:24:53] [INFO] Geometry optimization
[24-11-2023 01:24:53] [INFO] Method: M062X 6-311++G(d,p) NMR
[24-11-2023 01:28:23] [INFO] Identified logfile to be in ORCA format
[24-11-2023 01:28:23] [INFO] Success: True
[24-11-2023 01:28:23] [INFO] Num Iter: 9
[24-11-2023 01:28:23] [INFO] Elapsed Time: 209.9s


CPU times: user 122 ms, sys: 29 ms, total: 151 ms
Wall time: 3min 29s


In [25]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 01:28:58] [INFO] Geometry optimization
[24-11-2023 01:28:58] [INFO] Method: M062X cc-pVDZ NMR
[24-11-2023 01:31:09] [INFO] Identified logfile to be in ORCA format
[24-11-2023 01:31:09] [INFO] Success: True
[24-11-2023 01:31:09] [INFO] Num Iter: 9
[24-11-2023 01:31:09] [INFO] Elapsed Time: 130.3s


CPU times: user 156 ms, sys: 2.9 ms, total: 159 ms
Wall time: 2min 10s


In [26]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 01:31:09] [INFO] Geometry optimization
[24-11-2023 01:31:09] [INFO] Method: M062X cc-pVTZ NMR
[24-11-2023 01:36:34] [INFO] Identified logfile to be in ORCA format
[24-11-2023 01:36:34] [INFO] Success: True
[24-11-2023 01:36:34] [INFO] Num Iter: 9
[24-11-2023 01:36:34] [INFO] Elapsed Time: 325.4s


CPU times: user 180 ms, sys: 21.8 ms, total: 202 ms
Wall time: 5min 25s


In [27]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 01:36:34] [INFO] Geometry optimization
[24-11-2023 01:36:34] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 01:52:49] [INFO] Identified logfile to be in ORCA format
[24-11-2023 01:52:49] [INFO] Success: True
[24-11-2023 01:52:49] [INFO] Num Iter: 9
[24-11-2023 01:52:49] [INFO] Elapsed Time: 975.1s


CPU times: user 214 ms, sys: 75.5 ms, total: 290 ms
Wall time: 16min 15s


In [ ]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X aug-cc-pVTZ OPT NMR'
orca.geometry_optimization(conf)

In [ ]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

In [ ]:
data.scfvalues

In [ ]:
data.scftargets

In [ ]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

In [ ]:
dir(data)

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
len(data.converged_geometries)

In [ ]:
dir(data)

In [ ]:
data.optdone

In [ ]:
data.metadata

In [ ]:
data.new_geometries

In [ ]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
data.natom, data.nmo

In [ ]:
data.metadata

In [ ]:
data.scfenergies